In [1]:
from torch.utils.data import Dataset

class StateTransitionDataset(Dataset):
    def __init__(self, pool):
        self.pool = pool

    def __len__(self):
        return len(self.pool)

    def __getitem__(self, idx):
        state, action, reward, next_state, terminated = self.pool[idx]
        return state, next_state

In [6]:
from utils import *
from RobotEnv import RobotEnv
import numpy as np
model = torch.nn.Sequential(
    torch.nn.Linear(8, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 4),
)
env = RobotEnv(400,400)
# 玩一局游戏获取轨迹
controller = Controller(model,env)
t_list = []
s, _ = env.reset()


def get_trajectory(model,env):
    t = []
    terminated = False
    truncated = False
    while not (terminated or truncated):
        a = model(torch.FloatTensor(s).reshape(
            1, 8)).argmax().item()
        if random.random() < 0.1:
            a = env.action_space.sample()
        ns, r, terminated, truncated, _ = env.step(a)
        t.append(ns)
    return np.array(t)

In [9]:
t_1 = get_trajectory(model,env)

KeyboardInterrupt: 

In [ ]:
# 获取多条轨迹，存储至t_list
for i in range(100):
    t = get_trajectory(model,env)
    t_list.append(t)